# Functions

Functions in C++ require a **return type** and a **body**. It may return a *variable*, value or *nothing* (`void`). A void return type means that assigning the function output to a variable results in an erorr. This behavior is different from Python which allows assignment of `None` which is the default return value whenever none is specified.

In [1]:
from utils import *

In [2]:
%%runcpp
#include <iostream>
using namespace std;

int add_one(int x) {
    return x + 1;
}

void hello(){
    cout << "hello\n";
}

int main() {
    cout << add_one(3) << endl;
    hello();
    return 0;
}

g++ -std=c++23 ./code/tmp.cpp -o ./code/tmp
./code/tmp

4
hello


## Newton's method

**Example.** Let's define an actual meaningful function. Recall **Newton's Method** for getting the zero of $f$: 

$$x_{k+1} = x_k - \frac{f(x)}{f^\prime(x)}$$

Given a number $a > 0$ we want to solve for its square root using $f(x) = x^2 - a.$

$$
x_{k + 1} = x_k - \frac{x_k^2 - a}{2 x_k} = \frac{1}{2} \left( x_k + \frac{a}{x_k} \right).
$$

In [3]:
%%runcpp 04_newton_sqrt.cpp --run=false
#include <iostream>
#include <iomanip>
using namespace std;

double squareroot(float a, int n_steps = 10) {
    int i;
    double x = a / 2;
    cout << fixed << setprecision(10);
    
    for (i = 0; i < n_steps; i++) {
        x = 0.5 * (x + a / x);
        cout << x << "\t" << "| err: " << x * x - a << endl;
    }
    
    return x;
}

int main() {
    double a; cin >> a;
    double x;
    x = squareroot(a);
    cout << "\noutput: " << x;
    return 0;
}

g++ -std=c++23 ./code/04_newton_sqrt.cpp -o ./code/04_newton_sqrt



**NOTE:** Here we use `<iomanip>` for `std::fixed` and `std::setprecision`. To increase `cout` precision from 6 to 8;

In [4]:
!echo 1011.25 | ./code/04_newton_sqrt

253.8125000000	| err: 63409.5351562500
128.8983701674	| err: 15603.5398318240
68.3718491123	| err: 3663.4597510372
41.5811465161	| err: 717.7417455967
32.9505313728	| err: 74.4875177519
31.8202382539	| err: 1.2775625347
31.8001635687	| err: 0.0004029930
31.8001572323	| err: 0.0000000000
31.8001572323	| err: -0.0000000000
31.8001572323	| err: -0.0000000000

output: 31.8001572323

In [5]:
round(1011.25 ** 0.5, 10)

31.8001572323

## Parameter passing

Calling a function by value involves copying the contents of the arguments into the memory locations of the corresponding **formal parameters**. If the function changes the values of the parameters, the original contents in the memory referenced by the arguments of the calling function do not change. This is analogous to local variables in Python functions.


In C++, we can pass a reference to a location in memory which allows changing the value in that location. To let the compiler know that you intend to use **pass by reference**, you attach an `&` to the end of the type name in the formal parameter list in the function declaration and header.

In [6]:
%%runcpp
#include <iostream>
using namespace std;

void add_one(int& x) {
    x = x + 1;
}

int main() {
    int u = 41; 
    add_one(u);
    cout << u << endl;
    return 0;
}

g++ -std=c++23 ./code/tmp.cpp -o ./code/tmp
./code/tmp

42


**Remark.** Removing the `&` in `add_one` results in `u` being the same.

## Arrays as Parameters in Functions

An array is a collection data type that is the ancestor of the Python list. We will discuss arrays in more detail in the next chapter. Functions can be used with array parameters to maintain a structured design. However, a formal parameter for an array is neither a call-by-value nor a call-by-reference, but a new type of parameter pass called an **array parameter**:

```c++
float f(int arr[]) {
    ...
} 
```

Moreover, an array is defined using the same notation.

In [7]:
%%runcpp
#include <iostream>
using namespace std;

double avg(int arr[], int n) {
    double s = 0;
    for (int i = 0; i < n; i++) {
        s += arr[i];
    }
    return s / n;
}

int main() {
    int arr[] = {1, 2};
    cout << avg(arr, 2) << endl;
    return 0;
}

g++ -std=c++23 ./code/tmp.cpp -o ./code/tmp
./code/tmp

1.5


**NOTE:** The functions with array parameters do not make private copies of the arrays. Instead, the reference is passed to reduce the impact on memory. Arrays can therefore always be permanently changed when passed as arguments to functions. ⚠️



In [8]:
%%runcpp
#include <iostream>
using namespace std;

void zero(int arr[]) {
    arr[0] = 0;
}

int main() {
    int arr[] = {1, 2};
    println("{}, {}", arr[0], arr[1]);
    zero(arr);
    println("{}, {}", arr[0], arr[1]);
    return 0;
}

g++ -std=c++23 ./code/tmp.cpp -o ./code/tmp
./code/tmp

1, 2
0, 2


To prevent ourselves from accidentally modifying any of these arrays, we can add the modifier `const` in the function head. The compiler will then raise an error if any statement within the function's definition modifies the elements of the `const` array:

In [9]:
!rm ./code/tmp

In [10]:
%%runcpp --exitcode=true
#include <iostream>
using namespace std;

void zero(const int arr[]) {
    arr[0] = 0;
}

int main() {
    int arr[] = {1, 2};
    println("{}, {}", arr[0], arr[1]);
    zero(arr);
    println("{}, {}", arr[0], arr[1]);
    return 0;
}

g++ -std=c++23 ./code/tmp.cpp -o ./code/tmp
./code/tmp

127


./code/tmp.cpp:5:12: error: read-only variable is not assignable
    5 |     arr[0] = 0;
      |     ~~~~~~ ^
1 error generated.
/bin/sh: line 1: ./code/tmp: No such file or directory


## Function Overloading

**Function overloading** is a unique feature of C++ (not present in Python) where functions of the same name but with different implementations can be defined. The functions are distinguished based on their parameters. Note that assigning different return types result in a warning:

In [11]:
%%runcpp
#include <iostream>
using namespace std;

void f(int n){
    println("1 params: {}", n);
}

void f(int n, int m){
    println("2 params: {}, {}", n, m);
}

int main() {
    f(4);
    f(5, 6);
    f(0);
    return 0;
}

g++ -std=c++23 ./code/tmp.cpp -o ./code/tmp
./code/tmp

1 params: 4
2 params: 5, 6
1 params: 0
